In [ ]:
%%capture
# install module to read xlsx files
!pip install ipyfilechooser

In [ ]:
# add file-chooser to open file

from ipyfilechooser import FileChooser

# Create and display a FileChooser widget
fc = FileChooser('./data-files')
fc.filter_pattern = [ '*.xls', '*.xlsx' ]
display(fc)

In [ ]:
# read file and insert column with index

import pandas as pd

import os
  
# save file basename without extension in variable to store results with this name
path_name, extension= os.path.splitext(fc.selected)
file_name= path_name.split('/')
data_file_name= file_name[-1]


# read selected Excel file
print('reading file: ' + fc.selected + '\n')
# Excel Sheet Name is 'Data'
df= pd.read_excel(fc.selected, 'Data')
#df= pd.read_excel('', 'Tabelle1', parse_dates= ['Ueberschrift1'])
#df.rename(columns= { 'Unnamed: 0': 'Date' }, inplace= True)
# drop 2nd row
#df= df.drop([0])
# set index to 'Date' column
#df= df.set_index('Date')

# insert column with a simple count index for fitting
#df.insert(0, 'Range', range(0, len(df)))

df.info()
print('\n')
df.head()

In [ ]:
%%capture

# install module to group checkboxes in a panel
!pip install panel==0.13.1

In [ ]:
# display checkboxes to select coulumns to delete

import panel as pn
# activate gui extensions in notebook
pn.extension()


# create button to toggle all checkboxes at once
toggle_all_delete= pn.widgets.Toggle(name='select all')
display(toggle_all_delete)
print('\n')


# create a checkbox for each data column
column_delete_group= pn.widgets.CheckBoxGroup(
    name='column_xdelete_group', value= [], options= list(df.columns),
    inline= False)

# function is called if button changes
def select_all_delete_fn(onoff):
    if onoff.new == True:
        column_delete_group.value= column_delete_group.options
    else:
        column_delete_group.value= []

# watch state changes of button
toggle_all_delete.param.watch(select_all_delete_fn, 'value')

print('Select fields to DELETE from list below:\n')
column_delete_group

In [ ]:
if len(column_delete_group.value) > 0:
    print('Deleting following fields:\n')
    for column in column_delete_group.value:
        print(column)
        df= df.drop(column, axis='columns')

In [ ]:
# display checkboxes to select columns to analyze

import panel as pn
pn.extension()

# create button to toggle all checkboxes at once
toggle_all_analyze= pn.widgets.Toggle(name='select all')
display(toggle_all_analyze)
print('\n')

column_checkbox_group= pn.widgets.CheckBoxGroup(
    name='column_analyze_group', value= [], options= list(df.columns),
    inline= False)

def select_all_analyze_fn(onoff):
    if onoff.new == True:
        column_checkbox_group.value= column_checkbox_group.options
    else:
        column_checkbox_group.value= []

toggle_all_analyze.param.watch(select_all_analyze_fn, 'value')

print('Select fields to analyze from list below:\n')
column_checkbox_group

In [ ]:
if len(column_checkbox_group.value) == 0:
    print('Go back and select at least one field to analyze!')
else:
    print('Selected fields to analyze:\n')
    for value in column_checkbox_group.value:
        print(value)

In [ ]:
# display checkboxes to select date column

import panel as pn

pn.extension()

field_options= list(df.columns)
field_options.insert(0, df.index.name)

column_date_group= pn.widgets.RadioBoxGroup(
    name='column_date_group', value= [], options= list(field_options),
    inline= False)

print('Select Date field from list below:\n')
column_date_group


In [ ]:
date_format_options= list(['YYYY (%Y)', 'mmYYYY (%m%Y)', 'mm.YYYY (%m.%Y)', 'mm-YYYY (%m-%Y)', 'mm/YYYY (%m/%Y)', 'YYYYmm (%Y%m)', 'YYYY.mm (%Y.%m)', 'YYYY-mm (%Y-%m)', 'YYYY/mm (%Y/%m)', 'ddmmYYYY (%d%m%Y)', 'dd.mm.YYYY (%d.%m.%Y)', 'dd-mm-YYYY (%d-%m-%Y)', 'dd/mm/YYYY (%d/%m/%Y)', 'mmddYYYY (%m%d%Y)', 'mm.dd.YYYY (%m.%d.%Y)', 'mm-dd-YYYY (%m-%d-%Y)', 'mm/dd/YYYY (%m/%d/%Y)', 'YYYYmmdd (%Y%m%d)', 'YYYY.mm.dd (%Y.%m.%d)', 'YYYY-mm-dd (%Y-%m-%d)', 'YYYY/mm/dd (%Y/%m/%d)', 'YYYYddmm (%Y%d%m)', 'YYYY.dd.mm (%Y.%d.%m)', 'YYYY-dd-mm (%Y-%d-%m)', 'YYYY/dd/mm (%Y/%d/%m)'])

date_format_group= pn.widgets.RadioBoxGroup(
    name='date_format_group', value= [], options= list(date_format_options),
    inline= False)

print('Select date format from list below:\n')
date_format_group


In [ ]:
import numpy as np

if column_date_group.value:
    date_format= date_format_group.value.split('(')[1].split(')')[0]
    print('Selected date field:')
    print(column_date_group.value, '  format: ' + date_format + '\n')

    if column_date_group.value in df:
        if df[column_date_group.value].dtypes != 'datetime64[ns]':
            print('column is not of type datetime64[ns], trying to convert...')
            df[column_date_group.value]= pd.to_datetime(df[column_date_group.value], format= date_format)

        df.set_index(column_date_group.value, inplace = True)
        #df= df.assign(datetime_diff=pd.to_datetime(df[column_date_group.value]).diff())

df.info()
print('\n')
df.head()

In [ ]:
# insert a year column to group by
df['year'] = df.index.to_series().dt.year

In [ ]:
# display checkboxes to select groupby coulumn

import panel as pn

pn.extension()

groupby_field_options= list(df.columns)
groupby_field_options.insert(0, df.index.name)

groupby_checkbox_group= pn.widgets.CheckBoxGroup(
    name='groupby_checkbox_group', value= [], options= list(groupby_field_options),
    inline= False)

print('Select fields to group data from below:\n')
groupby_checkbox_group

In [ ]:
# save mean and standard deviation of grouped data to Excel file

if len(groupby_checkbox_group.value) > 0:
    df_grouped= df.reset_index().groupby(by= groupby_checkbox_group.value)
else:
    df_grouped= df

import os
os.makedirs('plots', exist_ok= True)
out_file= 'plots/' + data_file_name + '_stats.xlsx'
with pd.ExcelWriter(out_file) as writer:
    df_grouped.mean().to_excel(writer, sheet_name= 'mean')
    df_grouped.std().to_excel(writer, sheet_name= 'std')
    
print('Datei gespeichert')

In [ ]:
# check for Gaussian distribution and print histograms 

import matplotlib.pyplot as plt
import scipy.stats as stats

colors=  [ 'blue', 'green', 'red', 'cyan', 'magenta', 'salmon', 'lawngreen', 'dodgerblue', 'blueviolet', 'grey', 'darkorange', 'darkgoldenrod', 'lime', 'cornflowerblue', 'blueviolet', 'brown', 'gold', 'teal', 'olive' ]
ci= 0
pi= 0
fig_cols= 3


print('{:<55}\t{:<6}\t{:<12}\t{:<12}\t{:<12}\n'.format('Name', 'Wert', 'D’Agostino K^2', 'Shapiro-Wilk', 'Anderson-Darling'))

# calculate number of rows in figure
fig_rows= len(column_checkbox_group.value) // fig_cols
if len(column_checkbox_group.value) % fig_cols != 0:
    fig_rows= fig_rows + 1
fig, ax= plt.subplots(fig_rows, fig_cols, figsize= (25, fig_rows * 10))

# calculate several tests of Gaussian distribution
for column in column_checkbox_group.value:
    # drop missing values from dataframe and convert to values array
    col_clean= df[column].dropna().to_numpy()

    # D’Agostino’s K^2 Test
    k2, p_k2= stats.normaltest(col_clean, axis= None)
    alpha = 0.05
    if p_k2 > alpha:
        k2_gauss= 'ja'
    else:
        k2_gauss= 'nein'

    # Shapiro-Wilk Test
    sw, p_sw= stats.shapiro(col_clean)
    if p_sw > alpha:
        sw_gauss= 'ja'
    else:
        sw_gauss= 'nein'

    ad_result= stats.anderson(col_clean)
    ad= ad_result.statistic
    for i in range(len(ad_result.critical_values)):
        if ad_result.significance_level[i] != 1.0:
            continue
        ad_sl, ad_cv = ad_result.significance_level[i], ad_result.critical_values[i]
        if ad_result.statistic < ad_result.critical_values[i]:
            ad_gauss= 'ja'
        else:
            ad_gauss= 'nein'

    print('{:<55}\t{:<6}\t{:<10}\t{:<10}\t{:<10}'.format(column, 'p', '{:.5E}'.format(p_k2), '{:.5E}'.format(p_sw),  ''))
    print('{:<55}\t{:<6}\t{:<12}\t{:<12}\t{:<12}'.format('', 'stat', '{:.5E}'.format(k2), '{:.5E}'.format(sw), '{:.5E}'.format(ad)))
    print('{:<55}\t{:<6}\t{:<12}\t{:<12}\t{:<12}'.format('', 'Gauss', k2_gauss , sw_gauss, ad_gauss))

    if fig_rows == 1:
        ax[pi % fig_cols].hist(col_clean, 32, color= colors[ci], label= column)
        ax[pi % fig_cols].set_title(column)
    else:
        ax[pi // fig_cols][pi % fig_cols].hist(col_clean, 32, color= colors[ci], label= column)
        ax[pi // fig_cols][pi % fig_cols].set_title(column)

    pi= (pi + 1)
    ci= (ci + 1) % len(colors)

print('\nHistogramme\n')
fig.show()

import os
os.makedirs('plots', exist_ok= True)
fig.savefig('plots/' + data_file_name + '_histograms.png', dpi= 300)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

ci= 0
pi= 0

# set number of rows in figure
fig, ax= plt.subplots(len(column_checkbox_group.value), 1, figsize= (25, fig_rows * 20))

# calculate mean, max, min and std grouped by date index
df_date_group= df.replace(0, np.NaN).groupby(column_date_group.value)
#df_zero_clean= df
#for column in column_checkbox_group.value:
#    if (df_zero_clean[column] == 0).all() == False:
#        df_zero_clean[column].replace(0, np.NaN, inplace=True)
#df_date_group= df_zero_clean.groupby(column_date_group.value)

df_date_group_mean= df_date_group.mean()
df_date_group_std= df_date_group.std()
df_date_group_min= df_date_group.min()
df_date_group_max= df_date_group.max()

print('\nTrends calculated with linear regression:\n')
print('{:<55}\t{:<16}\n'.format('Name', 'Trend'))

# run least squares linear regression with two different libraries to check if everything is fine
for column in column_checkbox_group.value:
    # drop missing values from dataframe
    #col_clean= df[column].dropna()
    df_date_group_mean[column].fillna(value= 0, inplace= True)
    col_clean= df_date_group_mean[column].dropna()
    # run regression method 1 with numpy polynomial fit with order 1
    coeffs_polyfit= np.polyfit(range(0, len(col_clean)), col_clean, 1)
    # create matrix from values and run regression method 2 with numpy linear leastsquares
    a= np.vstack([range(0, len(col_clean)), np.ones(len(col_clean))]).T
    coeffs_lstsq= np.linalg.lstsq(a, col_clean, None)[0]
    # print to check if results of polyfit and lstsq are equal
    #print(coeffs_polyfit, '\n', coeffs_lstsq)
    # print result table line
    print('{:<55}\t{:+.6f}'.format(column, coeffs_polyfit[0]))

    # create y values to plot trend
    yn = np.polyval(coeffs_polyfit, range(0, len(col_clean)))
    # plot values and trend
    if len(column_checkbox_group.value) == 1:
        #ax.scatter(df_date_group_mean.index, df_date_group_mean[column], c= colors[ci], label= column)
        ax.plot(df_date_group_mean.index, yn, c= colors[ci])
        trend_patch= mpatches.Patch(color= colors[ci], label= 'Trend: ' + '{:.6f}'.format(coeffs_polyfit[0]))
        ci= (ci + 1) % len(colors)
        ax.errorbar(df_date_group_mean.index, df_date_group_mean[column], yerr= df_date_group_std[column], marker= 'o', c= colors[ci], linestyle= 'none', capsize= 5, elinewidth= 3, solid_capstyle= 'butt')
        error_patch= mpatches.Patch(color= colors[ci], label= column)
        ax.legend(handles= [ error_patch, trend_patch ], fontsize=18)
    else:
        #ax[pi].scatter(df_date_group_mean.index, df_date_group_mean[column], c= colors[ci], label= column)
        ax[pi].plot(df_date_group_mean.index, yn, c= colors[ci])
        trend_patch= mpatches.Patch(color= colors[ci], label= 'Trend: ' + '{:.6f}'.format(coeffs_polyfit[0]))
        ci= (ci + 1) % len(colors)
        ax[pi].errorbar(df_date_group_mean.index, df_date_group_mean[column], yerr= [ df_date_group_mean[column] - df_date_group_min[column], df_date_group_max[column] - df_date_group_mean[column] ], marker= 'o', c= colors[ci], linestyle= 'none', capsize= 5, elinewidth= 3, solid_capstyle= 'butt')
        error_patch= mpatches.Patch(color= colors[ci], label= column)
        ax[pi].legend(handles= [ error_patch, trend_patch ], fontsize=18)
    pi= (pi + 1)
    ci= (ci + 1) % len(colors)

print('\n')
fig.show()
fig.savefig('plots/' + data_file_name + '_trends_linear.png', dpi= 300)

In [ ]:
%%capture

# install pandas_profiling module
!pip install pandas_profiling

In [ ]:
import pandas_profiling

pfrep= pandas_profiling.ProfileReport(df)
pfrep.to_file('plots/' + data_file_name + '_profile_report.html')

In [ ]:
# show correlation

corr= df.corr()
corr.style.background_gradient(cmap= 'coolwarm').format(precision= 2)

In [ ]:
%%capture

# install Mann Kendall test module
!pip install pymannkendall

import pymannkendall as mk
 
mann_kendall= pd.DataFrame(index= [ 'trend', 'h', 'p', 'z', 'Tau', 's', 'var_s', 'slope', 'intercept' ])

# perform Mann-Kendall test
for column in column_checkbox_group.value:
    mann_kendall[column]= mk.original_test(df_date_group_mean[column].dropna())

In [ ]:
mann_kendall.head(10)

In [ ]:
print('\nTrends calculated with Mann-Kendall Test:\n')

print('{:<55}\t{:<12}\n'.format('Name', 'Mann-Kendall'))
# run least squares linear regression with two different libraries to check if everything is fine
for column in column_checkbox_group.value:
    print('{:<55}\t{:+.6f}'.format(column, mann_kendall[column].slope))
